In [1]:
import nltk
nltk.download('punkt')
from nltk import *
import numpy as np
from urllib import request
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import svm
from scipy.optimize import curve_fit
from PyDictionary import PyDictionary
from vocabulary.vocabulary import Vocabulary as vb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Meet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
url = "http://www.gutenberg.org/files/11/11-0.txt"  # Corpus used: Alice in Wonderland
res = request.urlopen(url)
text = res.read().decode('utf8')
# text= text.lower()
# loading the corpus from the Ebook itself hence NOTE that there might be some extra text in the corpus too

In [3]:
sent_tokenize_list = sent_tokenize(text)
word_tokenize_list = word_tokenize(text)
#tweet_tokenizer = TweetTokenizer()
#tweet_tokenizer_list = tweet_tokenizer.tokenize(text)

In [4]:
import string
translate_table_1 = dict((ord(char), None) for char in string.punctuation)   
translate_table_2 = dict((ord(char), " ") for char in string.whitespace[1:])

sent_tokenize_filtered_list_og = [(sent_tokenize_list[i].translate(translate_table_1)).translate(translate_table_2)
                          for i in range(2,len(sent_tokenize_list)-117)]
sent_tokenize_filtered_list_og = [" SsSs "+sent_tokenize_filtered_list_og[i]+" EeEe " for i in range(len(sent_tokenize_filtered_list_og))]

In [5]:
import sklearn
sent_tokenize_filtered_list, sent_tokenize_filtered_list_test = sklearn.model_selection.train_test_split(
    sent_tokenize_filtered_list_og, test_size = 0.2)

In [6]:
len(sent_tokenize_filtered_list)

779

In [7]:
mod_text = "".join(sent_tokenize_filtered_list)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens = word_tokenize(mod_text)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens = [tokens[i].translate(translate_table_3) for i in range(len(tokens))]
for i in range(len(tokens)):
    if tokens[i].isalpha()==False:
        tokens[i]=""
tokens = [value for value in tokens if value != ""]
# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

In [8]:
from collections import Counter
unigrams = list(nltk.ngrams(tokens, 1))
bigrams = list(nltk.ngrams(tokens, 2))
trigrams = list(nltk.ngrams(tokens, 3))
quadgrams = list(nltk.ngrams(tokens, 4))

In [9]:
count_unigrams = Counter(unigrams)
count_bigrams = Counter(bigrams)
count_trigrams = Counter(trigrams)
count_quadgrams = Counter(quadgrams)
no_of_tokens = len(tokens)

In [10]:
prob_unigrams = dict()
for i in range(len(unigrams)):
    prob_unigrams[unigrams[i]] = count_unigrams[unigrams[i]]/no_of_tokens 
    
prob_bigrams = dict()
for i in range(len(bigrams)):
    prob_bigrams[bigrams[i]] = count_bigrams[bigrams[i]]/count_unigrams[(bigrams[i][0],)] 

prob_trigrams = dict()
for i in range(len(trigrams)):
    prob_trigrams[trigrams[i]] = count_trigrams[trigrams[i]]/count_bigrams[(trigrams[i][0],trigrams[i][1])] 
    
prob_quadgrams = dict()
for i in range(len(quadgrams)):
    prob_quadgrams[quadgrams[i]] = count_quadgrams[quadgrams[i]]/count_trigrams[(quadgrams[i][0],quadgrams[i][1],quadgrams[i][2])] 

In [11]:
def Probability(sentence, model_name):  # in logspace
    
    sentence = ' SsSs '+sentence+' EeEe '
    words = word_tokenize(sentence)
    len_words = len(words)
    
    if model_name==unigrams:
        result_prob = 0
        for i in range(len(words)):
            result_prob = result_prob + prob_unigrams[(words[i],)]
    
    if model_name==bigrams:
        result_prob = 0 #prob_unigrams[(words[0],)]
        for i in range(1,len(words)):
            result_prob = result_prob + prob_bigrams[(words[i-1],words[i])]
    
    if model_name==trigrams:
        result_prob = prob_bigrams[(words[0],words[1])] #prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])]
        for i in range(2,len(words)):
            result_prob = result_prob + prob_trigrams[(words[i-2],words[i-1],words[i])]
            
    if model_name==quadgrams:
        result_prob = prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]#prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]
        for i in range(3,len(words)):
            result_prob = result_prob + prob_quadgrams[(words[i-3],words[i-2],words[i-1],words[i])]
    
    return result_prob
        

In [12]:
def Generator(model_name):
    
    import operator
    import random
    prob_unigrams_sorted = dict(sorted(prob_unigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_bigrams_sorted = dict(sorted(prob_bigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_trigrams_sorted = dict(sorted(prob_trigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_quadgrams_sorted = dict(sorted(prob_quadgrams.items(), key=operator.itemgetter(1))[::-1])
    
    if model_name==bigrams:
        sentence=["SsSs"]
        word=sentence[0]
        counter = 0
        while word!="EeEe" and counter<=25:
            candidate_bigrams = []
            candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
            for i in range(len(candidate_bigrams_keys)):
                if candidate_bigrams_keys[i][0]==word:
                    candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
            if word==sentence[0]:
                word = random.choice(candidate_bigrams)[1]
            else:
#                result = random.choice(candidate_bigrams)
                maximum = 0.0
                for i in range(len(candidate_bigrams)):
                    if prob_bigrams_sorted[candidate_bigrams[i]]>maximum:
                        result=candidate_bigrams[i]
                word = result[1]
            sentence = sentence + [word]
            counter = counter + 1
            #print(sentence)
        if counter>25:
            sentence = sorted(set(sentence),key = sentence.index)
        return " ".join(sentence)
    
    if model_name==trigrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_bigrams = []
        candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
        for i in range(len(candidate_bigrams_keys)):
            if candidate_bigrams_keys[i][0]==word:
                candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
        if word==sentence[0]:
            word = random.choice(candidate_bigrams)[1]
        sentence = sentence + [word]
        while word!="EeEe":
            candidate_trigrams = []
            candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
            for i in range(len(candidate_trigrams_keys)):
                if candidate_trigrams_keys[i][0]==sentence[-2] and candidate_trigrams_keys[i][1]==sentence[-1]:
                    candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
#            result = random.choice(candidate_trigrams)
            maximum = 0.0
            for i in range(len(candidate_trigrams)):
                if prob_trigrams_sorted[candidate_trigrams[i]]>maximum:
                    result=candidate_trigrams[i]
            word = result[2]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==quadgrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_trigrams = []
        candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
        for i in range(len(candidate_trigrams_keys)):
            if candidate_trigrams_keys[i][0]==word:
                candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
        if word==sentence[0]:
            trigram = random.choice(candidate_trigrams)
        sentence = sentence + [trigram[1]] + [trigram[2]]
        while word!="EeEe":
            candidate_quadgrams = []
            candidate_quadgrams_keys = list(prob_quadgrams_sorted.keys())
            for i in range(len(candidate_quadgrams_keys)):
                if candidate_quadgrams_keys[i][0]==sentence[-3] and candidate_quadgrams_keys[i][1]==sentence[-2] and candidate_quadgrams_keys[i][2]==sentence[-1]:
                    candidate_quadgrams = candidate_quadgrams + [candidate_quadgrams_keys[i]]
#            result = random.choice(candidate_quadgrams) 
            maximum = 0.0
            for i in range(len(candidate_quadgrams)):
                if prob_quadgrams_sorted[candidate_quadgrams[i]]>maximum:
                    result=candidate_quadgrams[i]
            word = result[3]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==unigrams:
        sentence=["SsSs"]
        word=sentence[0]
        while word!="EeEe":
            candidate_unigrams_keys = list(prob_unigrams_sorted.keys())
            result = random.choice(candidate_unigrams_keys)
            word=result[0]
            sentence=sentence + [word]
        return " ".join(sentence)
        

In [14]:
Generator(quadgrams)

'SsSs Of course they were said the Dormouse well in This answer so confused poor Alice that she let the Dormouse go on for some time without interrupting it EeEe'

In [15]:
uniq_tokens = sorted(set(tokens),key = tokens.index)
voc_len = len(uniq_tokens)
count_bigrams_smoothed = dict() 
for (first,second) in prob_bigrams:
    count_bigrams_smoothed[(first,second)] = ((count_bigrams[(first,second)] + 1) * count_unigrams[(first,)])/(count_unigrams[(first,)] + voc_len)

In [16]:
count_bigrams_diff=dict()
for k, v in count_bigrams.items():
    count_bigrams_diff[k] = v - count_bigrams_smoothed.get(k, 0)

In [17]:
count_bigrams_diff

{('SsSs', 'I'): 74.53933541607498,
 ('I', 'll'): 17.210626185958255,
 ('ll', 'try'): 0.9719525350593312,
 ('try', 'and'): 1.9912727272727273,
 ('and', 'say'): 0.6443778110944527,
 ('say', 'How'): 0.9705353934602946,
 ('How', 'doth'): 1.977198697068404,
 ('doth', 'the'): 1.9978134110787171,
 ('the', 'little'): 7.449922958397535,
 ('little', 'and'): 1.8974991194082422,
 ('and', 'she'): 26.132233883058472,
 ('she', 'crossed'): 0.7420584498094027,
 ('crossed', 'her'): 0.9985422740524781,
 ('her', 'hands'): 5.53079278666213,
 ('hands', 'on'): 0.9912854030501089,
 ('on', 'her'): 3.747229916897507,
 ('her', 'lap'): 0.86594079618918,
 ('lap', 'as'): 0.9985422740524781,
 ('as', 'if'): 11.1162474507138,
 ('if', 'she'): 7.800998573466477,
 ('she', 'were'): 1.6130876747141043,
 ('were', 'saying'): 1.928444286222855,
 ('saying', 'lessons'): 0.9927325581395349,
 ('lessons', 'and'): 0.9927325581395349,
 ('and', 'began'): 7.221889055472264,
 ('began', 'to'): 6.870828848223897,
 ('to', 'repeat'): 2.315

In [18]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
num = 11
for i in range(1,num+1):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

for i in range(1,num):
    good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]
    

In [19]:
good_turing_count_diff=dict()
for k, v in good_turing_count_smoothed.items():
    good_turing_count_diff[k] = k - v

In [20]:
good_turing_count_smoothed

{1: 0.2900639399167766,
 2: 1.0643806857942617,
 3: 1.893491124260355,
 4: 3.0833333333333335,
 5: 4.824324324324325,
 6: 3.823529411764706,
 7: 5.415384615384616,
 8: 8.181818181818182,
 9: 8.5,
 10: 7.764705882352941}

In [21]:
good_turing_N

{1: 9853,
 2: 1429,
 3: 507,
 4: 240,
 5: 148,
 6: 119,
 7: 65,
 8: 44,
 9: 40,
 10: 34,
 11: 24}

In [22]:
good_turing_count_diff

{1: 0.7099360600832234,
 2: 0.9356193142057383,
 3: 1.106508875739645,
 4: 0.9166666666666665,
 5: 0.17567567567567544,
 6: 2.176470588235294,
 7: 1.5846153846153843,
 8: -0.18181818181818166,
 9: 0.5,
 10: 2.235294117647059}

In [23]:
# uniq_tokens = sorted(set(tokens),key = tokens.index)
# voc_len = len(uniq_tokens)
# count_bigrams_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = ((count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1) * count_unigrams[(uniq_tokens[i],)])/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = count_unigrams[(uniq_tokens[i],)]/(count_unigrams[(uniq_tokens[i],)] + voc_len)
             

In [24]:
# mod_text_og = "".join(sent_tokenize_filtered_list_og)
# # tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
# tokens_og = word_tokenize(mod_text_og)
# uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
# voc_len = len(uniq_tokens)
# prob_bigrams_add1_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = (count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1)/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = 1/(count_unigrams[(uniq_tokens[i],)] + voc_len)
            

In [25]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
voc_len = len(uniq_tokens)

In [26]:
#result_prob_add1_smoothed = 1
perplexity_add1 = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] + 1)/(count_unigrams[(words[j],)] + voc_len)
        else:
            int_prob = 1/(count_unigrams[(words[j],)] + voc_len)
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_add1 = perplexity_add1 + [perp] 

0.0054858934169279 0.0054858934169279
7.0671734839650885e-06 0.0012882447665056361
4.6025226206220055e-09 0.0006512536633018561
8.922499426084662e-12 0.0019386106623586429
5.368531543973925e-15 0.0006016847172081829
1.75844465901537e-18 0.00032754667540124465
1.541046962517382e-20 0.008763693270735524
3.668286033128736e-24 0.00023803856224708403
1.2031111948601952e-27 0.00032797638570022957
1.788746944484382e-30 0.0014867677668748143
2.2969463171549048e-32 0.012841091492776886
1.437838070206513e-35 0.0006259780907668231
4.706507594784004e-39 0.0003273322422258593
0.0054858934169279 0.0054858934169279
1.7667933709912721e-06 0.00032206119162640903
5.7214811236764e-10 0.0003238341968911917
1.8600393770079324e-13 0.00032509752925877764
5.940719824362608e-17 0.00031938677738741617
5.79394001075677e-20 0.0009752925877763329
4.0247340161552095e-22 0.006946454413892909
3.536673124916704e-24 0.008787346221441126
3.717664695624752e-26 0.010511756569847857
5.961617536280872e-29 0.0016035920461834

0.002612330198537095 0.002612330198537095
8.537026792604885e-07 0.000326797385620915
2.7999431920645735e-10 0.00032797638570022957
8.860579721723334e-14 0.00031645569620253165
2.906060912339565e-17 0.00032797638570022957
0.0005224660397074191 0.0005224660397074191
3.424883904997831e-07 0.0006555227794165847
3.766876653832636e-09 0.010998552821997106
1.135970040359661e-12 0.00030156815440289503
2.968886090453214e-15 0.0026135249918327343
8.212686280645128e-19 0.00027662517289073305
2.6935671632158504e-22 0.00032797638570022957
4.215285075455165e-25 0.001564945226917058
1.3497550673887816e-28 0.0003202049311559398
2.570350045015533e-31 0.0019043084979766722
8.408079964067821e-35 0.00032711808963035657
2.4665162867213295e-37 0.0029335071707953064
5.871259906501618e-41 0.00023803856224708403
1.9250032480333173e-44 0.0003278688524590164
1.1217967645881803e-47 0.0005827505827505828
3.675611941638861e-51 0.000327653997378768
5.928406357482033e-54 0.0016129032258064516
4.233567977254487e-57 0.

0.0073145245559038665 0.0073145245559038665
2.3640997271828913e-05 0.003232062055591467
7.129371915509322e-09 0.00030156815440289503
9.090687810659002e-12 0.001275103602167676
8.874735903669705e-15 0.0009762447120078099
2.90308665478237e-18 0.00032711808963035657
3.560977190778743e-21 0.0012266176019625882
3.347831893555697e-24 0.000940144155437167
1.0897890278501618e-27 0.0003255208333333333
2.5941181334209992e-30 0.00238038562247084
1.6966109440294305e-33 0.0006540222367560497
5.553554644940853e-37 0.0003273322422258593
1.8101547082597303e-40 0.0003259452411994785
1.150034757471239e-43 0.0006353240152477764
3.7693699032161227e-47 0.00032776138970829236
1.0349725159846576e-50 0.0002745744096650192
3.392240301490192e-54 0.00032776138970829236
0.00522466039707419 0.00522466039707419
5.084002981259348e-06 0.0009730781706130393
8.242546986477543e-09 0.001621271076523995
2.4797072763169505e-12 0.00030084235860409147
1.6233762856412116e-15 0.0006546644844517185
4.730117382404463e-19 0.00029

ZeroDivisionError: float division by zero

In [27]:
import statistics as sts
perplexity_add1_mean = sts.mean(perplexity_add1)

In [28]:
perplexity_add1_mean

852.9276462541335

In [29]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
for i in set(count_bigrams_values):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

# # for i in range(1,11):
# #     good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]

In [30]:
good_turing_N

{1: 9853,
 2: 1429,
 3: 507,
 4: 240,
 5: 148,
 6: 119,
 7: 65,
 8: 44,
 9: 40,
 778: 1,
 11: 24,
 12: 15,
 13: 16,
 10: 34,
 15: 8,
 14: 17,
 17: 6,
 18: 9,
 19: 4,
 20: 5,
 16: 12,
 22: 4,
 23: 3,
 24: 5,
 21: 4,
 25: 2,
 26: 3,
 27: 2,
 29: 3,
 30: 1,
 32: 2,
 36: 1,
 37: 2,
 166: 1,
 39: 3,
 42: 7,
 43: 1,
 45: 2,
 46: 1,
 47: 1,
 48: 1,
 52: 2,
 54: 1,
 60: 1,
 75: 1,
 88: 1,
 91: 1,
 96: 1}

In [31]:
d = 1
i = 10
while (d >= 1.0 or d < 0.0):
    d = sts.median(list(good_turing_count_diff.values())[:i])
    i = i-1

In [32]:
d

0.9261429904362024

In [33]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens_og = [tokens_og[i].translate(translate_table_3) for i in range(len(tokens_og))]
for i in range(len(tokens_og)):
    if tokens_og[i].isalpha()==False:
        tokens_og[i]=""
tokens_og = [value for value in tokens_og if value != ""]
# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

from collections import Counter
unigrams_og = list(nltk.ngrams(tokens_og, 1))
bigrams_og = list(nltk.ngrams(tokens_og, 2))
trigrams_og = list(nltk.ngrams(tokens_og, 3))
quadgrams_og = list(nltk.ngrams(tokens_og, 4))

count_unigrams_og = Counter(unigrams_og)
count_bigrams_og = Counter(bigrams_og)
count_trigrams_og = Counter(trigrams_og)
count_quadgrams_og = Counter(quadgrams_og)
no_of_tokens_og = len(tokens_og)

prob_unigrams_og = dict()
for i in range(len(unigrams_og)):
    prob_unigrams_og[unigrams_og[i]] = count_unigrams_og[unigrams_og[i]]/no_of_tokens_og 
    
prob_bigrams_og = dict()
for i in range(len(bigrams_og)):
    prob_bigrams_og[bigrams_og[i]] = count_bigrams_og[bigrams_og[i]]/count_unigrams_og[(bigrams_og[i][0],)] 

prob_trigrams_og = dict()
for i in range(len(trigrams_og)):
    prob_trigrams_og[trigrams_og[i]] = count_trigrams_og[trigrams_og[i]]/count_bigrams_og[(trigrams_og[i][0],trigrams_og[i][1])] 
    
prob_quadgrams_og = dict()
for i in range(len(quadgrams_og)):
    prob_quadgrams_og[quadgrams_og[i]] = count_quadgrams_og[quadgrams_og[i]]/count_trigrams_og[(quadgrams_og[i][0],quadgrams_og[i][1],quadgrams_og[i][2])] 



In [34]:
#result_prob_add1_smoothed = 1
perplexity_good_turing = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] - d)/(count_unigrams[(words[j],)])
        else:
            int_prob = (d/count_unigrams_og[(words[j],)])*prob_unigrams_og[(words[j+1],)]
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_good_turing = perplexity_good_turing + [perp] 

0.024485053927553017 0.024485053927553017
0.0009067589413786324 0.03703316088506781
3.0441138093391196e-06 0.003357136798354436
2.6959313869752445e-07 0.08856210890356081
7.240488372042645e-11 0.0002685709438683549
6.26725203135302e-14 0.0008655841580455352
1.1192563925232664e-14 0.17858806170934108
2.3747104006650846e-22 2.1216858054404372e-08
3.0071374910772475e-24 0.012663175645480978
2.9437932024426396e-26 0.009789353533642667
1.6982054765988933e-26 0.5768766213570272
8.590710830579308e-30 0.0005058699285191616
3.3530776102156755e-32 0.0039031433793349593
0.024485053927553017 0.024485053927553017
5.389266115393604e-07 2.2010431879543735e-05
1.539930743092017e-11 2.8574034202791418e-05
1.155936828798039e-15 7.506420882779708e-05
1.5826768856121925e-20 1.3691724722170886e-05
6.2946987684787165e-22 0.039772481835696205
3.422371441716171e-23 0.054369105934886204
2.6323059311877943e-24 0.07691467673853096
1.6776930700900206e-25 0.06373472969887596
7.473896526656257e-27 0.044548652312518

1.5328670844263204e-238 0.00016029336260102502
5.660648945725264e-240 0.0369285047818988
3.6030355553180005e-247 6.365057416321312e-08
6.81500367984909e-250 0.001891461678692095
5.1537601914148025e-252 0.007562373306787307
4.3738639371075105e-259 8.486743221761749e-08
8.62354670387135e-262 0.001971608359992608
6.29338607543003e-267 7.297909191591383e-06
3.6028143434898134e-273 5.724762950036609e-07
5.6595708142628284e-275 0.01570874953490045
1.2866857958658538e-275 0.2273468851424642
2.729942989150414e-283 2.1216858054404372e-08
8.524287123251462e-285 0.031225147034679674
0.010364386405088315 0.010364386405088315
1.8459359424088395e-08 1.7810373622336115e-06
8.284955821934539e-11 0.004488214152828701
1.897176325284038e-17 2.2899051800146435e-07
5.923960970770736e-19 0.031225147034679674
9.481002511398921e-05 9.481002511398921e-05
3.5011924657938957e-06 0.0369285047818988
3.1108747875372186e-07 0.08885186455557585
3.966554616222308e-13 1.2750608388717901e-06
2.0076904632966254e-13 0.506

1.4012496121702062e-138 0.11877641165014836
3.8761088392255654e-142 0.0002766180133475565
6.584716053940077e-144 0.01698795448493052
3.6459102014955795e-145 0.05536928504781899
6.563451760079656e-148 0.0018002230985796853
3.0059364368365266e-153 4.579810360029287e-06
9.63663318450941e-158 3.205867252020501e-05
2.211993699010217e-159 0.022954009524466783
1.5921106070359937e-161 0.007197627225377734
1.0586271643507617e-165 6.649206152338816e-05
2.4241558273510225e-171 2.2899051800146435e-06
1.262872289363637e-174 0.0005209534284533313
3.0019294059055755e-175 0.23770649108297812
5.704629422610371e-177 0.01900320977364718
1.1780685177757742e-178 0.020651096337765337
2.636624781338612e-181 0.0022380911989029573
1.913363061070222e-182 0.07256865196036007
5.395783905606557e-184 0.02820052302352108
1.5874795531855139e-186 0.002942073998804925
0.00137850707261078 0.00137850707261078
5.09062050227885e-05 0.0369285047818988
2.684001824721838e-07 0.005272445320801907
2.9295978936217926e-08 0.10915

0.0 0.0369285047818988
0.0 0.0055782090185156705
0.0 0.02342757444084604
0.0 2.0036670325128126e-06
0.0 1.60293362601025e-05
0.0 0.007651336508155594
0.0 4.477468229078912e-08
0.0 0.005546150345995466
0.0 0.015711037951240892
0.0 1.8858042658944122e-07
0.0 0.0012182295557677902
0.0 1.2654739152712502e-06
0.0 0.010408382344893224


ZeroDivisionError: float division by zero

In [35]:
import statistics as sts
perplexity_good_turing_mean = sts.mean(perplexity_good_turing)

In [36]:
perplexity_good_turing_mean

717.4997434847069